In [2]:
import pandas as pd

# ETF 데이터 로드
etf = pd.read_parquet('./important_data/high_low_data_ETF.parquet')

# 날짜 데이터를 datetime 형식으로 변환
etf['date'] = pd.to_datetime(etf['date'])

# 2009년 이후 데이터만 사용
# etf = etf[etf['date']]

def calculate_and_merge_returns(df, ticker, freq):
    df_filtered = df[df['TICKER'] == ticker].copy()
    df_filtered['date'] = pd.to_datetime(df_filtered['date'])
    df_filtered.set_index('date', inplace=True)

    # 수익률 계산
    ret_col = 'vwretx' if ticker == 'SPY' else 'ewretx'
    next_ret_col = 'nxt_freq_vwretx' if ticker == 'SPY' else 'nxt_freq_ewret'
    
    # 주간 데이터를 'W'로 리샘플링하고 각 주의 마지막 거래일을 사용
    if freq == 'W':
        df_resampled = df_filtered['Adj Close'].resample('W-FRI').last().pct_change()
    elif freq == 'ME':
        df_resampled = df_filtered['Adj Close'].resample('ME').last().pct_change()
    elif freq == 'QE':
        df_resampled = df_filtered['Adj Close'].resample('QE').last().pct_change()
    elif freq == 'YE':
        df_resampled = df_filtered['Adj Close'].resample('YE').last().pct_change()
    else:
        raise ValueError(f"Unsupported frequency: {freq}")
    
    df_resampled = df_resampled.dropna().reset_index()
    df_resampled.rename(columns={'Adj Close': ret_col}, inplace=True)
    df_resampled[next_ret_col] = df_resampled[ret_col].shift(-1)
    
    return df_resampled[['date', ret_col, next_ret_col]].dropna()

def save_merged_returns(etf, freq):
    """
    SPY와 RSP의 수익률 데이터를 병합하고 빈도별로 파일로 저장하는 함수.
    """
    spy_returns = calculate_and_merge_returns(etf, 'SPY', freq)
    rsp_returns = calculate_and_merge_returns(etf, 'RSP', freq)
    
    merged_returns = pd.merge(spy_returns, rsp_returns, on='date', suffixes=('_vw', '_ew'))
    
    if freq == 'W': filename = f'spy_week_ret.csv'
    if freq == 'ME': filename = f'spy_month_ret.csv'
    if freq == 'QE': filename = f'spy_quarter_ret.csv'
    if freq == 'YE': filename = f'spy_year_ret.csv'
    
    merged_returns.to_csv(filename, index=False)
    print(f'File saved: {filename}')

# 각 빈도별로 수익률 계산 및 파일 저장
for freq in ['W', 'ME', 'QE', 'YE']:

    save_merged_returns(etf, freq)

File saved: spy_week_ret.csv
File saved: spy_month_ret.csv
File saved: spy_quarter_ret.csv
File saved: spy_year_ret.csv


In [2]:
# us_month_ret.pq 열기
import pandas as pd

path = '/home/indi/codespace/PriceTrend/CACHE_DIR/us_month_ret.pq'

us_month_ret = pd.read_parquet(path)
us_month_ret.to_csv('us_month_ret.csv', index=False)

In [3]:
us_month_ret

,Date,StockID,MarketCap,next_month_ret,next_month_ret_0delay,Ret_5d,Ret_6-20d,Ret_6-60d
0,1993-01-29,10001,1.512000e+04,0.017857,0.017857,3.240260e-07,0.035714,0.065641
1,1993-01-29,10002,1.369900e+04,0.105263,0.105263,7.894773e-02,0.024390,0.073172
2,1993-01-29,10003,1.042150e+04,-0.078948,-0.078948,-1.052639e-01,0.029412,-0.176469
3,1993-01-29,10009,1.505400e+04,-0.009616,-0.009616,-4.845446e-02,0.040817,0.224494
4,1993-01-29,10010,9.537600e+04,-0.054374,-0.054374,-2.083338e-02,-0.012552,-0.218724
...,...,...,...,...,...,...,...,...
2223505,2019-12-31,93423,3.812878e+06,-0.610460,-0.610460,-3.436065e-02,-0.130165,-0.676398
2223506,2019-12-31,93426,4.249320e+05,-0.251469,-0.251469,-2.294129e-02,0.043650,-0.419625
2223507,2019-12-31,93427,2.399275e+06,0.120136,0.120136,1.897023e-02,0.006809,-0.268355
2223508,2019-12-31,93434,6.994050e+04,0.109526,0.109526,2.380888e-02,0.046510,-0.023255
